Setup libraries

In [18]:
import nltk
nltk.download('punkt')  

[nltk_data] Downloading package punkt to /home/jlam/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [1]:
import sqlite3

conn = sqlite3.connect("../server/myelin.db")
cur = conn.cursor()
result = cur.execute("select * from resources")
print(result)

In [4]:
import pandas as pd
import sqlite3

conn = sqlite3.connect("../server/myelin.db")
df = pd.read_sql("select * from resources", conn)
print(df)

                                                 uri  \
0  https://medium.learningbyshipping.com/bicycle-...   

                                           title  \
0  “Bicycle for the Mind” - Learning By Shipping   

                                                text  \
0  “When we invented the personal computer, we cr...   

                                              markup markup_type  
0  <div id="readability-page-1" class="page"><div...        html  


In [1]:
import nltk
from sentence_transformers import SentenceTransformer, util
import numpy as np
from lexrank import LexRank, degree_centrality_scores

In [2]:
model = SentenceTransformer('all-MiniLM-L6-v2')

Downloading:   0%|          | 0.00/1.18k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/10.2k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/612 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/116 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/39.3k [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/350 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/13.2k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [13]:
import sqlite3
conn = sqlite3.connect("../server/myelin.db")
cur = conn.cursor()
res = cur.execute("SELECT text FROM resources")
doc = None
for row in res:
    doc = row[0]

In [19]:
print(f"characters in doc: {len(doc)}")
sentences = nltk.sent_tokenize(doc)
print(f"sentences in doc: {len(sentences)}")


characters in doc: 15901
sentences in doc: 111


In [25]:
embeddings = model.encode(sentences, convert_to_tensor=True)
print(f"Dimensions of the embeddings array: {embeddings.size()}")

Dimensions of the embeddings array: torch.Size([111, 384])


## Core algorithm

Compute the pair-wise cosine similarity scores and rank them

In [27]:
cos_scores = util.cos_sim(embeddings, embeddings).cpu().numpy()
centrality_scores = degree_centrality_scores(cos_scores, threshold=None)
most_central_sentence_indices = np.argsort(-centrality_scores)
for idx in most_central_sentence_indices[0:5]:
    print(sentences[idx].strip())

“When we invented the personal computer, we created a new kind of bicycle…a new man-machine partnership…a new generation of entrepreneurs.” Steve Jobs said this and a lot more in 1980 as explored in this annotated twitter thread.1/ “When we invented the personal computer, we created a new kind of bicycle…a new man-machine partnership…a new generation of entrepreneurs.” — Steve Jobs, c. 1980Such a remarkable quote and articulation.
!The “personal computer” world was diverse, with a dozen or so companies making thousands of units and maybe a hundred other companies trying stuff out.
Scan from youtube.19/ “Computers are such a part of life that many people believe computers don’t invade their privacy…life, liberty, and pursuit of happiness is inevitably linked to a computer.”How’s that for east coast optimism?20/ Ken Kashiwahara then reports “Computers are being welcomed in schools, work, and homes without any mention of invasion of privacy” and discusses all the popular home uses.Jobs ad